In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.stats import skew
from tqdm import tqdm
import pandas as pd
from google.colab import drive

In [ ]:
# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Tentukan Path untuk Input dan Output
input_path = "/content/drive/MyDrive//Result_Preprocessed(fix)"
output_csv = "/content/drive/MyDrive/Ekstraksi_GLCM&LBP_(fix).csv"

# 3. Folder kategori
classes = ['Berkualitas', 'Kurang Berkualitas', 'Tidak Berkualitas']

# 4. Fungsi untuk mendapatkan usia simpan berdasarkan nama file
def get_storage_age(filename):
    if filename.startswith('0_'):
        return '0 minggu'
    elif filename.startswith('1_'):
        return '1 minggu'
    elif filename.startswith('2_'):
        return '2 minggu'
    elif filename.startswith('3_'):
        return '3 minggu'
    elif filename.startswith('4_'):
        return '4 minggu'
    elif filename.startswith('5_'):
        return '5 minggu'
    else:
        return 'Tidak diketahui'

Mounted at /content/drive


In [ ]:
# Fungsi untuk Gamma Correction (untuk meningkatkan kontras gambar)
def gamma_correction(image, gamma=1.5):
    # Menggunakan gamma correction untuk meningkatkan kontras gambar
    gamma_corrected = np.array(255 * (image / 255) ** gamma, dtype='uint8')
    return gamma_corrected

# Fungsi untuk normalisasi gambar (Quantization)
def normalize_image(image, levels=8):
    image = np.float32(image)
    normalized_image = cv2.normalize(image, None, 0, levels - 1, cv2.NORM_MINMAX)
    return np.uint8(normalized_image)

# Fungsi untuk Histogram Equalization (CLAHE)
def apply_clahe(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(gray)
    return equalized_image

# Fungsi untuk Z-Score Normalization pada fitur
def z_score_normalization(features):
    return (features - np.mean(features)) / np.std(features)

In [ ]:
# ====== Ekstraksi Fitur GLCM ======
def extract_glcm_features(image, levels=8):
    # Terapkan CLAHE dan Gamma correction untuk memperbaiki distribusi
    equalized_image = apply_clahe(image)
    corrected_image = gamma_correction(equalized_image)
    quantized_image = normalize_image(corrected_image, levels=levels)

    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    distances = [1]

    glcm = graycomatrix(quantized_image, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    glcm = np.squeeze(glcm)

    contrast, correlation, energy, homogeneity = 0, 0, 0, 0
    for i in range(len(angles)):
        contrast += np.sum((np.arange(levels)[:, None] - np.arange(levels))**2 * glcm[:,:,i])
        mean_x = np.mean(np.arange(levels)[:, None] * glcm[:,:,i], axis=0)
        mean_y = np.mean(np.arange(levels) * glcm[:,:,i], axis=1)
        correlation += np.sum((np.arange(levels)[:, None] - mean_x) * (np.arange(levels) - mean_y) * glcm[:,:,i])
        energy += np.sum(glcm[:,:,i]**2)
        homogeneity += np.sum(glcm[:,:,i] / (1 + np.abs(np.arange(levels)[:, None] - np.arange(levels))))

    contrast /= len(angles)
    correlation /= len(angles)
    energy /= len(angles)
    homogeneity /= len(angles)

    entropy = -np.mean([np.sum(glcm[:,:,i] * np.log2(glcm[:,:,i] + 1e-12)) for i in range(len(angles))])

    variance_glcm = np.mean([np.sum((np.arange(levels)[:, None] - np.mean(quantized_image))**2 * glcm[:,:,i]) for i in range(len(angles))])
    std_dev_glcm = np.sqrt(variance_glcm)

    skewness_glcm = skew(quantized_image.flatten())

    mean_glcm_features = np.mean([contrast, correlation, energy, homogeneity, entropy, variance_glcm, std_dev_glcm, skewness_glcm])

    return contrast, correlation, energy, homogeneity, entropy, variance_glcm, std_dev_glcm, skewness_glcm, mean_glcm_features


In [ ]:
# ====== Ekstraksi Fitur LBP ======
def extract_lbp_features(image, P=8, R=1, levels=8):
    equalized_image = apply_clahe(image)
    corrected_image = gamma_correction(equalized_image)
    quantized_image = normalize_image(corrected_image, levels=levels)

    lbp = local_binary_pattern(quantized_image, P, R, method='uniform')

    lbp_histogram, _ = np.histogram(lbp.ravel(), bins=np.arange(0, P + 3), range=(0, P + 2))
    lbp_histogram = lbp_histogram.astype('float')
    lbp_histogram /= (lbp_histogram.sum() + 1e-6)

    lbp_energy = np.sum(lbp_histogram ** 2)
    lbp_entropy = -np.sum(lbp_histogram * np.log2(lbp_histogram + (lbp_histogram == 0)))
    lbp_contrast = np.sum((np.arange(0, P + 2) ** 2) * lbp_histogram)
    lbp_homogeneity = np.sum(lbp_histogram / (1 + np.abs(np.arange(0, P + 2)[:, None] - np.arange(0, P + 2))))
    lbp_variance = np.var(lbp_histogram)
    lbp_std_dev = np.std(lbp_histogram)

    lbp_contrast_norm = lbp_contrast / ((P + 2 - 1) ** 2) if lbp_contrast > 0 else 0
    lbp_variance_norm = lbp_variance / ((P + 2 - 1) / 2) ** 2 if lbp_variance > 0 else 0
    lbp_std_dev_norm = lbp_std_dev / ((P + 2 - 1) / 2) if lbp_std_dev > 0 else 0

    lbp_entropy_norm = lbp_entropy / np.log2(P + 2) if lbp_entropy > 0 else 0
    lbp_homogeneity_norm = lbp_homogeneity / 1.0 if lbp_homogeneity > 0 else 0

    skewness_lbp = skew(lbp_histogram)

    mean_lbp_features = np.mean([lbp_energy, lbp_entropy_norm, lbp_contrast_norm, lbp_homogeneity_norm, lbp_variance_norm, lbp_std_dev_norm, skewness_lbp])

    return lbp_energy, lbp_entropy_norm, lbp_contrast_norm, lbp_homogeneity_norm, lbp_variance_norm, lbp_std_dev_norm, skewness_lbp, mean_lbp_features


In [ ]:
# ====== Memproses Semua Gambar dan Menyimpan Fitur ke CSV ======
def process_and_extract_features(input_path, output_csv):
    columns = ['filename', 'contrast_glcm', 'correlation_glcm', 'energy_glcm', 'homogeneity_glcm', 'entropy_glcm', 'variance_glcm', 'std_dev_glcm', 'skewness_glcm', 'mean_glcm',
               'energy_lbp', 'entropy_lbp', 'contrast_lbp', 'homogeneity_lbp', 'variance_lbp', 'std_dev_lbp', 'skewness_lbp', 'mean_lbp', 'kelas_kualitas', 'usia_simpan']

    features_df = pd.DataFrame(columns=columns)

    classes = ['Berkualitas', 'Kurang Berkualitas', 'Tidak Berkualitas']

    for kls in classes:
        input_dir = os.path.join(input_path, kls)

        for filename in tqdm(os.listdir(input_dir), desc=f"Proses {kls}"):
            if filename.lower().endswith(".jpg"):
                path_img = os.path.join(input_dir, filename)
                img = cv2.imread(path_img)

                # Cek apakah gambar berhasil dibaca
                if img is None:
                    print(f"Gambar {filename} tidak dapat dibaca. Melewatkan gambar.")
                    continue  # Melewati gambar yang tidak bisa dibaca

                # Ekstraksi fitur GLCM (sudah dinormalisasi)
                glcm_features = extract_glcm_features(img)

                # Ekstraksi fitur LBP (sudah dinormalisasi)
                lbp_features = extract_lbp_features(img)

                # Menyimpan fitur yang diekstraksi ke dataframe
                base_name = os.path.splitext(filename)[0]
                analysis_row = pd.DataFrame([{
                    'filename': filename,
                    'contrast_glcm': glcm_features[0],
                    'correlation_glcm': glcm_features[1],
                    'energy_glcm': glcm_features[2],
                    'homogeneity_glcm': glcm_features[3],
                    'entropy_glcm': glcm_features[4],
                    'variance_glcm': glcm_features[5],
                    'std_dev_glcm': glcm_features[6],
                    'skewness_glcm': glcm_features[7],
                    'mean_glcm': glcm_features[8],
                    'energy_lbp': lbp_features[0],
                    'entropy_lbp': lbp_features[1],
                    'contrast_lbp': lbp_features[2],
                    'homogeneity_lbp': lbp_features[3],
                    'variance_lbp': lbp_features[4],
                    'std_dev_lbp': lbp_features[5],
                    'skewness_lbp': lbp_features[6],
                    'mean_lbp': lbp_features[7],
                    'kelas_kualitas': kls,
                    'usia_simpan': get_storage_age(filename)
                }])

                # Menggunakan concat untuk menambahkan baris
                features_df = pd.concat([features_df, analysis_row], ignore_index=True)

    # Menyimpan hasil ekstraksi fitur ke CSV
    features_df.to_csv(output_csv, index=False)

In [ ]:
# 9. Panggil proses untuk ekstraksi fitur dan simpan ke CSV
process_and_extract_features(input_path, output_csv)

Proses Berkualitas:   0%|          | 0/810 [00:00<?, ?it/s]/tmp/ipython-input-1280554066.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  features_df = pd.concat([features_df, analysis_row], ignore_index=True)
Proses Tidak Berkualitas: 100%|██████████| 810/810 [01:14<00:00, 10.82it/s]
